This notebook performs the synthesis using our proposed synthesization approach applied to the IPUMS data. The synthesis model is a CART. Logistic regression with three way interactions is used as the pMSE model.

In [12]:
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import itertools
import os
import pickle

from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

from sklearn.preprocessing import StandardScaler

from helper_functions import *

rng = np.random.RandomState(42)

Import the data.

In [13]:
# import standardized lat/long location data
train_data = pd.read_csv("../Data/IBM_Telco_Cleaned.csv")

In [14]:
train_data

,label,cat6,cat9,cat17,cat20,cat22,cat23
0,1,fbad5c96,a73ee510,8efede7f,a458ea53,ad3062eb,c7dc6720
1,0,fbad5c96,a73ee510,e5ba7672,a458ea53,c9d4222a,bcdee96c
2,0,fbad5c96,a73ee510,e5ba7672,a458ea53,c0061c6d,423fab69
3,0,7e0ccccf,a73ee510,3486227d,a458ea53,ad3062eb,c7dc6720
4,0,7e0ccccf,a73ee510,e5ba7672,5840adea,ad3062eb,32c7478e
...,...,...,...,...,...,...,...
10071,1,7e0ccccf,a73ee510,e5ba7672,a458ea53,c9d4222a,32c7478e
10072,0,7e0ccccf,a73ee510,e5ba7672,b1252a9d,ad3062eb,c7dc6720
10073,0,7e0ccccf,7cc72ec2,1e88c74f,5840adea,c9d4222a,bcdee96c
10074,0,fbad5c96,a73ee510,e5ba7672,a458ea53,c0061c6d,c7dc6720


***

Functions for logistic and multinomial logistic regression synthesizer.

In [8]:
def train_models_mn(#overall parameters
                    train_data,
                    number_synthetic_datasets,
                    # hyperparameters for GMM, end with underscore means Bayesian optimization will choose
                    number_gmm_initializations,
                    num_components_,
                    # hyperparameters for CART, end with underscore means Bayesian optimization will choose
                    C_gender,
                    C_SeniorCitizen,
                    C_Partner,
                    C_Dependents,
                    C_tenure,
                    C_PhoneService,
                    C_MultipleLines,
                    C_OnlineSecurity,
                    C_OnlineBackup,
                    C_DeviceProtection,
                    C_TechSupport,
                    C_StreamingTV,
                    C_StreamingMovies,
                    C_PaperlessBilling
):
    
    num_samples = train_data.shape[0]

    # normalized version of training data
    scaler = StandardScaler().fit(train_data.loc[:,['Churn']])
    norm_train = train_data.copy()
    norm_train.loc[:,['Churn']] = scaler.transform(norm_train.loc[:,['Churn']])
    
    ########## Code for GMM ############
    
    # fit GMM model
    GMM = GaussianMixture(num_components_, n_init=number_gmm_initializations, covariance_type='full', init_params="k-means++", random_state=rng).fit(norm_train.loc[:,['Churn']])
    
    # list for synthetic datasets
    sXs = []
    
    # generate and store number_synthetic_datasets synthetic datasets
    for i in range(number_synthetic_datasets):
        sX = GMM.sample(num_samples)[0]
        sX = pd.DataFrame(sX, columns=['Churn'])
        sXs.append(sX)
        
    ####################################################################################################
        
    ########### Code for gender MN ##########
    
    synth_gender_vars = multinomial_synthesizer(orig_data=norm_train.loc[:,['Churn']], 
                                                   synth_data_sets=sXs, 
                                                   target=norm_train.gender, 
                                                   penalty_param=C_gender)
    print("After synthesis:", synth_gender_vars[i].std())
    sXs = [pd.concat([Y, synth_gender_vars[i]], axis=1) for i,Y in enumerate(sXs)]  

    ####################################################################################################
        
    ########### Code for SeniorCitizen MN ##########
    
    synth_SeniorCitizen_vars = multinomial_synthesizer(orig_data=norm_train.loc[:,['gender', 'Churn']], 
                                             synth_data_sets=sXs, 
                                             target=norm_train.SeniorCitizen, 
                                             penalty_param=C_SeniorCitizen)
    print("After synthesis:", synth_SeniorCitizen_vars[i].std())
    sXs = [pd.concat([Y, synth_SeniorCitizen_vars[i]], axis=1) for i,Y in enumerate(sXs)]
    
    ####################################################################################################

    ########### Code for Partner MN ##########
    
    synth_Partner_vars = multinomial_synthesizer(orig_data=norm_train.loc[:,['SeniorCitizen', 'gender', 'Churn']], 
                                                   synth_data_sets=sXs, 
                                                   target=norm_train.Partner, 
                                                   penalty_param=C_Partner)
    print("After synthesis:", synth_Partner_vars[i].std())
    sXs = [pd.concat([Y, synth_Partner_vars[i]], axis=1) for i,Y in enumerate(sXs)]  
     
    ####################################################################################################

    ########### Code for Dependents MN ##########

    synth_Dependents_vars = multinomial_synthesizer(orig_data=norm_train.loc[:,['Partner', 'SeniorCitizen', 'gender', 'Churn']], 
                                                   synth_data_sets=sXs, 
                                                   target=norm_train.Dependents, 
                                                   penalty_param=C_Dependents)
    print("After synthesis:", synth_Dependents_vars[i].std())
    sXs = [pd.concat([Y, synth_Dependents_vars[i]], axis=1) for i,Y in enumerate(sXs)]  

    ####################################################################################################

    ########### Code for tenure MN ##########

    synth_tenure_vars = multinomial_synthesizer(orig_data=norm_train.loc[:,['Dependents', 'Partner', 'SeniorCitizen', 'gender', 'Churn']], 
                                                   synth_data_sets=sXs, 
                                                   target=norm_train.tenure, 
                                                   penalty_param=C_tenure)
    print("After synthesis:", synth_tenure_vars[i].std())
    sXs = [pd.concat([Y, synth_tenure_vars[i]], axis=1) for i,Y in enumerate(sXs)]  

    ####################################################################################################

    ########### Code for PhoneService MN ##########

    synth_PhoneService_vars = multinomial_synthesizer(orig_data=norm_train.loc[:,['tenure', 'Dependents', 'Partner', 'SeniorCitizen', 'gender', 'Churn']], 
                                                   synth_data_sets=sXs, 
                                                   target=norm_train.PhoneService, 
                                                   penalty_param=C_PhoneService)
    print("After synthesis:", synth_PhoneService_vars[i].std())
    sXs = [pd.concat([Y, synth_PhoneService_vars[i]], axis=1) for i,Y in enumerate(sXs)]  

    ####################################################################################################

    ########### Code for MultipleLines MN ##########

    synth_MultipleLines_vars = multinomial_synthesizer(orig_data=norm_train.loc[:,['PhoneService', 'tenure', 'Dependents', 'Partner', 'SeniorCitizen', 'gender', 'Churn']], 
                                                   synth_data_sets=sXs, 
                                                   target=norm_train.MultipleLines, 
                                                   penalty_param=C_MultipleLines)
    print("After synthesis:", synth_MultipleLines_vars[i].std())
    sXs = [pd.concat([Y, synth_MultipleLines_vars[i]], axis=1) for i,Y in enumerate(sXs)]  

    ####################################################################################################

    ########### Code for OnlineSecurity MN ##########

    synth_OnlineSecurity_vars = multinomial_synthesizer(orig_data=norm_train.loc[:,['MultipleLines', 'PhoneService', 'tenure', 'Dependents', 'Partner', 'SeniorCitizen', 'gender', 'Churn']], 
                                                   synth_data_sets=sXs, 
                                                   target=norm_train.OnlineSecurity, 
                                                   penalty_param=C_OnlineSecurity)
    print("After synthesis:", synth_OnlineSecurity_vars[i].std())
    sXs = [pd.concat([Y, synth_OnlineSecurity_vars[i]], axis=1) for i,Y in enumerate(sXs)]  

    ####################################################################################################

    ########### Code for OnlineBackup MN ##########

    synth_OnlineBackup_vars = multinomial_synthesizer(orig_data=norm_train.loc[:,['OnlineSecurity', 'MultipleLines', 'PhoneService', 'tenure', 'Dependents', 'Partner', 'SeniorCitizen', 'gender', 'Churn']], 
                                                   synth_data_sets=sXs, 
                                                   target=norm_train.OnlineBackup, 
                                                   penalty_param=C_OnlineBackup)
    print("After synthesis:", synth_OnlineBackup_vars[i].std())
    sXs = [pd.concat([Y, synth_OnlineBackup_vars[i]], axis=1) for i,Y in enumerate(sXs)]  

    ####################################################################################################

    ########### Code for DeviceProtection MN ##########

    synth_DeviceProtection_vars = multinomial_synthesizer(orig_data=norm_train.loc[:,['OnlineBackup', 'OnlineSecurity', 'MultipleLines', 'PhoneService', 'tenure', 'Dependents', 'Partner', 'SeniorCitizen', 'gender', 'Churn']], 
                                                   synth_data_sets=sXs, 
                                                   target=norm_train.DeviceProtection, 
                                                   penalty_param=C_DeviceProtection)
    print("After synthesis:", synth_DeviceProtection_vars[i].std())
    sXs = [pd.concat([Y, synth_DeviceProtection_vars[i]], axis=1) for i,Y in enumerate(sXs)]  

    ####################################################################################################

    ########### Code for TechSupport MN ##########

    synth_TechSupport_vars = multinomial_synthesizer(orig_data=norm_train.loc[:,['DeviceProtection', 'OnlineBackup', 'OnlineSecurity', 'MultipleLines', 'PhoneService', 'tenure', 'Dependents', 'Partner', 'SeniorCitizen', 'gender', 'Churn']], 
                                                   synth_data_sets=sXs, 
                                                   target=norm_train.TechSupport, 
                                                   penalty_param=C_TechSupport)
    print("After synthesis:", synth_TechSupport_vars[i].std())
    sXs = [pd.concat([Y, synth_TechSupport_vars[i]], axis=1) for i,Y in enumerate(sXs)]  

    ####################################################################################################

    ########### Code for StreamingTV MN ##########

    synth_StreamingTV_vars = multinomial_synthesizer(orig_data=norm_train.loc[:,['TechSupport', 'DeviceProtection', 'OnlineBackup', 'OnlineSecurity', 'MultipleLines', 'PhoneService', 'tenure', 'Dependents', 'Partner', 'SeniorCitizen', 'gender', 'Churn']], 
                                                   synth_data_sets=sXs, 
                                                   target=norm_train.StreamingTV, 
                                                   penalty_param=C_StreamingTV)
    print("After synthesis:", synth_StreamingTV_vars[i].std())
    sXs = [pd.concat([Y, synth_StreamingTV_vars[i]], axis=1) for i,Y in enumerate(sXs)]  

    ####################################################################################################

    ########### Code for StreamingMovies MN ##########

    synth_StreamingMovies_vars = multinomial_synthesizer(orig_data=norm_train.loc[:,['StreamingTV', 'TechSupport', 'DeviceProtection', 'OnlineBackup', 'OnlineSecurity', 'MultipleLines', 'PhoneService', 'tenure', 'Dependents', 'Partner', 'SeniorCitizen', 'gender', 'Churn']], 
                                                   synth_data_sets=sXs, 
                                                   target=norm_train.StreamingMovies, 
                                                   penalty_param=C_StreamingMovies)
    print("After synthesis:", synth_StreamingMovies_vars[i].std())
    sXs = [pd.concat([Y, synth_StreamingMovies_vars[i]], axis=1) for i,Y in enumerate(sXs)]  

    ####################################################################################################

    ########### Code for PaperlessBilling MN ##########

    synth_PaperlessBilling_vars = multinomial_synthesizer(orig_data=norm_train.loc[:,['StreamingMovies', 'StreamingTV', 'TechSupport', 'DeviceProtection', 'OnlineBackup', 'OnlineSecurity', 'MultipleLines', 'PhoneService', 'tenure', 'Dependents', 'Partner', 'SeniorCitizen', 'gender', 'Churn']], 
                                                   synth_data_sets=sXs, 
                                                   target=norm_train.PaperlessBilling, 
                                                   penalty_param=C_PaperlessBilling)
    print("After synthesis:", synth_PaperlessBilling_vars[i].std())
    sXs = [pd.concat([Y, synth_PaperlessBilling_vars[i]], axis=1) for i,Y in enumerate(sXs)]  

    ####################################################################################################

    ########### Code for MonthlyCharges MN ##########

    #synth_MonthlyCharges_vars = multinomial_synthesizer(orig_data=norm_train.loc[:,['PaperlessBilling', 'StreamingMovies', 'StreamingTV', 'TechSupport', 'DeviceProtection', 'OnlineBackup', 'OnlineSecurity', 'MultipleLines', 'PhoneService', 'tenure', 'Dependents', 'Partner', 'SeniorCitizen', 'gender', 'Churn']], 
   #                                                synth_data_sets=sXs, 
   #                                                target=norm_train.MonthlyCharges, 
   #                                               penalty_param=C_MonthlyCharges)
   # print("After synthesis:", synth_MonthlyCharges_vars[i].std())
    #sXs = [pd.concat([Y, synth_MonthlyCharges_vars[i]], axis=1) for i,Y in enumerate(sXs)]  

    ####################################################################################################

    ########### Code for TotalCharges MN ##########

    #synth_TotalCharges_vars = multinomial_synthesizer(orig_data=norm_train.loc[:,['MonthlyCharges', 'PaperlessBilling', 'StreamingMovies', 'StreamingTV', 'TechSupport', 'DeviceProtection', 'OnlineBackup', 'OnlineSecurity', 'MultipleLines', 'PhoneService', 'tenure', 'Dependents', 'Partner', 'SeniorCitizen', 'gender', 'Churn']], 
    #                                               synth_data_sets=sXs, 
   #                                                target=norm_train.TotalCharges, 
    #                                               penalty_param=C_TotalCharges)
    #print("After synthesis:", synth_TotalCharges_vars[i].std())
    #sXs = [pd.concat([Y, synth_TotalCharges_vars[i]], axis=1) for i,Y in enumerate(sXs)]  

    ####################################################################################################
        
    ###### Calculate pMSE ratios ######
    pmse_ratios = [pmse_ratio(norm_train, Y) for Y in sXs]

    # convert sXs to original scale (un-normalize?)
    for i, Z in enumerate(sXs):
        H = train_data.sample(frac=1.0, replace=True, ignore_index=True).loc[:,['Churn']]
        scaler = StandardScaler().fit(H)
        Z.loc[:,['Churn']] = scaler.inverse_transform(Z.loc[:,['Churn']])
    print("pmse_ratios:", pd.Series(pmse_ratios).isna().sum())
    print("sXs:", pd.Series(sXs).isna().sum())
    print("GMM:", pd.Series(GMM).isna().sum())
    return pmse_ratios, sXs, GMM

In [9]:
def optimize_models_mn(train_data,
                       number_synthetic_datasets,
                       number_gmm_initializations,
                       random_state):

    def evaluate_models(num_components_, 
                        C_gender,
                        C_SeniorCitizen,
                        C_Partner,
                        C_Dependents,
                        C_tenure,
                        C_PhoneService,
                        C_MultipleLines,
                        C_OnlineSecurity,
                        C_OnlineBackup,
                        C_DeviceProtection,
                        C_TechSupport,
                        C_StreamingTV,
                        C_StreamingMovies,
                        C_PaperlessBilling):

        pmse_ratios, _, _ = train_models_mn(train_data=train_data,
                                            number_synthetic_datasets=number_synthetic_datasets,
                                            number_gmm_initializations=number_gmm_initializations,
                                            num_components_=int(num_components_),
                                            C_gender = C_gender,
                        C_SeniorCitizen = C_SeniorCitizen,
                        C_Partner = C_Partner,
                        C_Dependents = C_Dependents,
                        C_tenure = C_tenure,
                        C_PhoneService = C_PhoneService,
                        C_MultipleLines = C_MultipleLines,
                        C_OnlineSecurity = C_OnlineSecurity,
                        C_OnlineBackup = C_OnlineBackup,
                        C_DeviceProtection = C_DeviceProtection,
                        C_TechSupport = C_TechSupport,
                        C_StreamingTV = C_StreamingTV,
                        C_StreamingMovies = C_StreamingMovies,
                        C_PaperlessBilling = C_PaperlessBilling)
        
        return -1 * ((1 - np.mean(pmse_ratios))**2)

    optimizer = BayesianOptimization(
        f=evaluate_models,
        pbounds={
            "num_components_": (10, 200.99),
            "C_gender": (0.001, 3),
            "C_SeniorCitizen": (0.001, 3),
            "C_Partner": (0.001, 3),
            "C_Dependents": (0.001, 3),
            "C_tenure": (0.001, 3),
            "C_PhoneService": (0.001, 3),
            "C_MultipleLines": (0.001, 3),
            "C_OnlineSecurity": (0.001, 3),
            "C_OnlineBackup": (0.001, 3),
            "C_DeviceProtection": (0.001, 3),
            "C_TechSupport": (0.001, 3),
            "C_StreamingTV": (0.001, 3),
            "C_StreamingMovies": (0.001, 3),
            "C_PaperlessBilling": (0.001, 3)
        },
        random_state=random_state)
    
    utility = UtilityFunction(kind="ei", xi=1e-02)
    optimizer.maximize(init_points=5, n_iter=25, acquisition_function=utility)
    print("Final Result: ", optimizer.max)
    return optimizer.max, optimizer

The default value for $\alpha = 1e-06$.

In [10]:
nsd = 10
ngi = 2
random_states = [np.random.RandomState(1006), np.random.RandomState(428)]

In [11]:
optimization_results = [optimize_models_mn(train_data=train_data, number_synthetic_datasets=nsd, number_gmm_initializations=ngi, random_state=r) for r in random_states]
print("done")

|   iter    |  target   | C_Depe... | C_Devi... | C_Mult... | C_Onli... | C_Onli... | C_Pape... | C_Partner | C_Phon... | C_Seni... | C_Stre... | C_Stre... | C_Tech... | C_gender  | C_tenure  | num_co... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


/var/folders/m_/j4nrbkts2t51lhpxncv3l1mr0000gn/T/ipykernel_1308/4086273274.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.60102348 -0.60102348  1.66382851 ... -0.60102348  1.66382851
 -0.60102348]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  norm_train.loc[:,['Churn']] = scaler.transform(norm_train.loc[:,['Churn']])
/Users/ethanbudge/tutorial-env/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(


After synthesis: 0.5000300111126067


/Users/ethanbudge/tutorial-env/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(


After synthesis: 0.3697789426902705


...

After synthesis: 0.4826503201943736


/Users/ethanbudge/tutorial-env/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(


After synthesis: 0.48448092653057684


/Users/ethanbudge/tutorial-env/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/ethanbudge/tutorial-env/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


After synthesis: 0.4925387009197418
pmse_ratios: 0
sXs: 0
GMM: 0
| 30        | -3.292e+0 | 0.8325    | 0.4242    | 0.7424    | 0.7377    | 2.844     | 1.928     | 2.061     | 1.44      | 2.565     | 0.009024  | 1.544     | 2.718     | 2.14      | 1.758     | 19.47     |
Final Result:  {'target': np.float64(-2533.295322813445), 'params': {'C_Dependents': np.float64(2.087884244334957), 'C_DeviceProtection': np.float64(1.6608343270560781), 'C_MultipleLines': np.float64(0.001010899731055634), 'C_OnlineBackup': np.float64(3.0), 'C_OnlineSecurity': np.float64(1.6053794512368083), 'C_PaperlessBilling': np.float64(0.018219961951766915), 'C_Partner': np.float64(2.195320691268981), 'C_PhoneService': np.float64(0.11539401049820197), 'C_SeniorCitizen': np.float64(0.29662768387110994), 'C_StreamingMovies': np.float64(0.17443527068941236), 'C_StreamingTV': np.float64(2.2775039477172916), 'C_TechSupport': np.float64(2.902675436347372), 'C_gender': np.float64(1.1096338604154674), 'C_tenure': np.float6

In [ ]:
optimization_results

In [ ]:
optimized_datasets = []

for r in random_states:
    best_result, optimizer = optimize_models_mn(train_data=train_data, number_synthetic_datasets=nsd, number_gmm_initializations=ngi, random_state=r)
    
    best_params = best_result['params']
    
    # Run train_models_mn again with best params to get the synthetic datasets
    pmse_ratios, sXs, GMM = train_models_mn(
        train_data=train_data,
        number_synthetic_datasets=nsd,
        number_gmm_initializations=ngi,
        num_components_=int(best_params['num_components_']),
        C_hiking_int=best_params['C_hiking_int'],
        C_sustain_int=best_params['C_sustain_int'],
        C_online_int=best_params['C_online_int']
    )
    
    optimized_datasets.append(sXs)


In [ ]:
optimized_datasets[0][0]

In [ ]:
optimized_datasets[0][0].to_csv('synthesized_data_part_dependent_aa.csv')

In [ ]:
run_targets = [np.minimum.accumulate(-i[1].space.target) for i in optimization_results]

In [ ]:
print(run_targets)

In [ ]:
fig = plt.figure()
plt.plot(run_targets[0])
plt.scatter(np.arange(len(run_targets[0])), run_targets[0], s=6)
plt.plot(run_targets[1])
plt.scatter(np.arange(len(run_targets[1])), run_targets[1], s=6)
plt.title("CPS ASEC Data - Objective Value for MNL Synthesis")
plt.xlabel("Iteration")
plt.ylabel("Minimum Objective Value")
fig.savefig('../Images/min_objective_mnl_ipums.pdf')
plt.show()

Choose the params that gave the best objective value across all random seeds.

In [ ]:
best_params = optimization_results[np.argmax([x[0]['target'] for x in optimization_results])][0]

In [ ]:
best_params

Generate 20 synthetic data sets.

On occassion, the synthesis models will produce a significantly different fit than what was observed during the optimization process. In these cases, retrain the models until a fit (judged by the pMSE ratio) consistent with the optimization results is observed.

In [ ]:
pmse_ratios, full_sXs, GMM = train_models_mn(train_data=train_data,
                                             number_synthetic_datasets=20,
                                             # hyperparameters for GMM
                                             number_gmm_initializations=ngi,
                                             num_components_=int(best_params['params']['num_components_']),
                                             # hyperparameters for CART, end with underscore means Bayesian optimization will choose
                                             C_non_white_=best_params['params']['C_non_white_'],
                                             C_sex_=best_params['params']['C_sex_'])

In [ ]:
np.mean(pmse_ratios)

In [ ]:
plt.violinplot(pmse_ratios)
plt.xlabel("Density")
plt.ylabel("pMSE Ratio")
plt.title("Distribution of pMSE Ratios")
plt.show()

# Save the synthetic datasets.

Save the GMM model.

In [ ]:
for i, sX in enumerate(full_sXs):
    sX.to_csv("../Data/IPUMS/Synthetic Datasets/gmm_and_mnl_" + str(i) + ".csv", index=False)

# save
with open('../Results/IPUMS/Models/gmm_and_mnl.pkl','wb') as f:
    pickle.dump(GMM,f)

In [ ]:
# full_sXs = [pd.read_csv("../Data/IPUMS/Synthetic Datasets/gmm_and_mnl_" + str(i) + ".csv") for i in range(20)]

# # load
# with open('../Results/IPUMS/Models/gmm_and_mnl.pkl', 'rb') as f:
#     GMM = pickle.load(f)

***

Now apply the attribute disclosure prevention algorithm.

We don't have a great baseline for the probability of being non-white, so we'll just use the proportion from the data.

In [ ]:
# prior = np.mean(train_data.non_white == 1)

In [ ]:
# prior

For our threshold, we select $c = 10$, i.e., we are allowing for a 10x increase in the probability of an adversary inferring the non-white status based on the synthetic data. This is a relatively large increase. For example, this means going from XXX under the prior to no more than XXX under the updated probability.

In [ ]:
# c = 3

We provide a range of $\delta$ values over which to evaluate and prevent attribute disclosure.

In [ ]:
# deltas = np.linspace(0.001, 0.1, 5)

In [ ]:
# ad_sXs = [attribute_disclosure_reduction(original_data=train_data, 
#                                          synthetic_data=X,
#                                          continuous_vars=['incwage', 'years_of_educ', 'potential_experience'],
#                                          categorical_vars=['sex'],
#                                          sensitive_var='non_white',
#                                          num_mixture_components=190,#int(best_params['params']['num_components_']),
#                                          deltas=deltas, 
#                                          c=c, 
#                                          prior_prob=prior) for X in full_sXs]

***

In [ ]:
# num_records = train_data.shape[0]

In [ ]:
# np.mean([Z.shape[0] - num_records for Z in ad_sXs])

***

In [ ]:
# for i, sX in enumerate(ad_sXs):
    
#     # normalize
#     sX.loc[:,['incwage', 'years_of_educ', 'potential_experience']] = StandardScaler().fit_transform(sX.loc[:,['incwage', 'years_of_educ', 'potential_experience']])
#     full_sXs[i].loc[:,['incwage', 'years_of_educ', 'potential_experience']] = StandardScaler().fit_transform(full_sXs[i].loc[:,['incwage', 'years_of_educ', 'potential_experience']])

#     # convert to original scale (un-normalize?)
#     H = train_data.sample(frac=1.0, replace=True, ignore_index=True).loc[:,['incwage', 'years_of_educ', 'potential_experience']]
#     scaler = StandardScaler().fit(H.loc[:, ['incwage', 'years_of_educ', 'potential_experience']])
#     sX.loc[:,['incwage', 'years_of_educ', 'potential_experience']] = scaler.inverse_transform(sX.loc[:,['incwage', 'years_of_educ', 'potential_experience']])
#     full_sXs[i].loc[:,['incwage', 'years_of_educ', 'potential_experience']] = scaler.inverse_transform(full_sXs[i].loc[:,['incwage', 'years_of_educ', 'potential_experience']])
#     sX.to_csv("../Data/IPUMS/Synthetic Datasets/ad_gmm_and_mnl_" + str(i) + ".csv", index=False)
#     full_sXs[i].to_csv("../Data/IPUMS/Synthetic Datasets/gmm_and_mnl_" + str(i) + ".csv", index=False)